In [ ]:
حد ضرر همیشه ثابت است
و مثلا ما میگیم اگه 5 درصد ضرر کردم خارج شو
اما عملکرد استاپ تریل اینطور است که میاد یه عددی میگیره و میگه اگه این مقدار رفتی تو سود با همون حد ضرر میام استاپ جابه جا میکنم

In [ ]:
from ta_calculat import runmain
from websocket import create_connection
import uuid
import requests
import threading
from threading import Thread
import time
import datetime
import os.path
import sys
import pandas as pd
import talib
import numpy as np
import talib as ta
from ohlcv import ohlcvs
import asyncio
import nest_asyncio
import multiprocessing
print(time.ctime())
#.......................................................................................................................
sellorbuy=runmain()   #محاسبه اندیکاتورها
#.......................................................................................................................

price=0
chprice=0
def runprice():
    global price
    global chprice
    url='https://api.kucoin.com/api/v1/bullet-public'
    token=json.loads(requests.post(url).text)['data']['token']
    ws=create_connection('wss://ws-api.kucoin.com/endpoint?token=%s&[connectId=%s]' % (token,uuid.uuid1()))

    msg = {"id":"%s" % (uuid.uuid1()),
        "type": "subscribe",
        "topic": "/contractMarket/ticker:XBTUSDTM",
        "response": True}
    msg=json.dumps(msg)
    ws.send(msg)
    i=0
    y=1
    while y==1:
        try:
            p = json.loads(ws.recv())#
            if "data" in p:
                if 'price' in p['data']:
                    price=p['data']['price']
                    chprice=price
#                     with open("price.txt", "w") as output:
#                         output.write(str(price)+'\n')

    #                 print(price)
            if i==30:
                msg = {"id":"%s" % (uuid.uuid1()),
            "type":"pong"
                      }
                msg=json.dumps(msg)
                ws.send(msg)
                i=0
            i+=1
        except Exception as error:
            chprice=0
            print('break connection with websocket')
            break
    return
thread=threading.Thread(target=runprice)
thread.start()

#.......................................................................................................................

def timedif():
    def parse_duration(s):
        return list(map(lambda x: int(x), s.split(":")))

    inp1 = time.ctime()[11:19]
    [h1, m1, s1] = parse_duration(inp1)
    d1 = datetime.timedelta(hours=h1, minutes=m1, seconds=s1)

    inp2 = pd.read_pickle("data_pairs_kucoin_future_1m.pkl").loc[0,'pair'].iloc[-1,1]
    [h2, m2, s2] = parse_duration(inp2)
    d2 = datetime.timedelta(hours=h2, minutes=m2, seconds=s2)

    res = d1 - d2
    return str(res)[0:4]

#.......................................................................................................................
class backtest():
    def __init__(self,datasignal=None,leverage=1,BUY=True,SELL=True,total_size=False,
                 trigger=None,Commission=None,sl=None,tp=None,value=None,stoptraill=None,size=1):
        self.order=[('type position')]
        self.leverage=leverage
        self.leveragevalue=value*self.leverage-value
        self.datasignal=datasignal
        #cash is all money without open orders
        self.controlorder=['sl_tp','stoptraill']
        self.trigger=trigger    # sl_tp  or  stoptraill
        self.size=size
        self.total_size=total_size
        self.gross=None
        self.profit=[('index','traillist','percent profit (%)','gain','type position')]
        self.lose=[('index','traillist','percent lose (%)','lose','type position')]
        self.cash=value*self.leverage
        self.actionprice=None
        self.lastprice=None
        self.value=value*self.leverage
        self.sl=sl
        self.tp=tp
        self.stoptraill=stoptraill
        self.pricestoptraill=None
        self.Commission=Commission
        self.typeposition=None
        self.BUY=BUY
        self.SELL=SELL
    def sizer(self):
        global price
        if self.total_size==True:
            sizer=self.cash/self.actionprice
        else:
            sizer=self.size
        return sizer

    def buy(self,i,j):
        global price
        print('run buy for',i,j)
#         print('\n','buy','\n')
        self.typeposition='buy'
        self.order.append((self.typeposition))
        self.pricestoptraill=self.actionprice
        self.cash=self.value-self.actionprice*self.size
        traillist=[]
        traillist.append(self.pricestoptraill)
        xy=1
        while xy==1:
            self.lastprice=price

            if self.trigger==self.controlorder[0]:

                if self.lastprice - self.Commission*self.lastprice<=(1-self.sl)*self.actionprice \
                    or self.lastprice - self.Commission*self.lastprice>=(1+self.tp)*self.actionprice:
                    
                    self.gross=((self.lastprice-self.actionprice)/self.actionprice)*100

                    comis=self.lastprice*self.size*self.Commission
                    gain=(self.lastprice*self.size-self.actionprice*self.size)-comis
                    
                    

                    self.value=self.value+gain

                    self.cash=self.cash+self.lastprice*self.size-comis


                    if self.gross>=0:
                        self.profit.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    #profit
                    break

            elif self.trigger==self.controlorder[1]:
                # وقتی کمیسیون را قرار میدهیم میگیم کل ضرر ما با کمیسیون بشه اون مقدار حد ضرر
                if self.lastprice - self.Commission*self.lastprice<=(1-(self.sl/self.leverage))*self.pricestoptraill:

                    self.gross=((self.lastprice-self.actionprice)/self.actionprice)*100
                    comis=self.lastprice*self.size*self.Commission
                    gain=(self.lastprice*self.size-self.actionprice*self.size)-comis

                    self.value=self.value+gain

                    self.cash=self.cash+self.lastprice*self.size-comis
                    if self.gross>=0:
                        self.profit.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            traillist,
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                          traillist,
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    print('finish buy',i,j)
                    #profit
                    break
                elif (self.lastprice-self.lastprice*self.Commission)/self.pricestoptraill>=\
                (1+(self.stoptraill/self.leverage)):
                    
                    print(i,j,'  ', (self.lastprice-self.lastprice*self.Commission)/self.pricestoptraill ,
                          ' >= ',(1+(self.stoptraill/self.leverage)))
                    self.pricestoptraill=self.lastprice
                    traillist.append(self.pricestoptraill)
#         print(self.cash,self.value)
        return 0

    def sell(self,i,j):
        global price
        print('run sell for',i,j)
#         print('\n','sell','\n')
        self.typeposition='sell'
        self.order.append((self.typeposition))
        self.pricestoptraill=self.actionprice
        self.cash=self.value-self.actionprice*self.size
        traillist=[]
        traillist.append(self.pricestoptraill)
        yx=1

        while yx==1:
            self.lastprice=price
#             print(i , j ,'  ', self.lastprice,'>=',(1+(self.sl/self.leverage))*self.pricestoptraill)
            if self.trigger==self.controlorder[0]:
                
                if self.lastprice + self.Commission*self.lastprice>=(1+self.sl)*self.actionprice \
                    or self.lastprice + self.Commission*self.lastprice<=(1-self.tp)*self.actionprice:
                    
                    self.gross=((self.actionprice-self.lastprice)/self.actionprice)*100

                    comis=((self.actionprice*self.size-self.lastprice*self.size)+\
                    self.actionprice*self.size)*self.Commission
                    
                    self.cash=self.cash+((self.actionprice*self.size-self.lastprice*self.size)+\
                    self.actionprice*self.size)-comis

                    gain=(self.actionprice*self.size-self.lastprice*self.size)-comis

                    self.value=self.value+gain

                    if self.gross>=0:
                        self.profit.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    break

            elif self.trigger==self.controlorder[1]:

                if self.lastprice + self.Commission*self.lastprice>=(1+(self.sl/self.leverage))*self.pricestoptraill:
                    
                    self.gross=((self.actionprice-self.lastprice)/self.actionprice)*100
                    comis=((self.actionprice*self.size-self.lastprice*self.size)+\
                    self.actionprice*self.size)*self.Commission
                    
                    self.cash=self.cash+((self.actionprice*self.size-self.lastprice*self.size)+\
                    self.actionprice*self.size)-comis

                    gain=(self.actionprice*self.size-self.lastprice*self.size)-comis

                    self.value=self.value+gain

                    if self.gross>=0:
                        self.profit.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            traillist,
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))

                    if self.gross<0:
                        self.lose.append(('start price:%s - end price:%s' % (self.actionprice,self.lastprice),
                                            traillist,
                                            float("{:.2f}".format(self.gross)),
                                            float("{:.2f}".format(gain)),self.typeposition))
                    print('finish sell',i,j)
                    break
                elif (self.lastprice+self.lastprice*self.Commission)/self.pricestoptraill<=\
                (1-(self.stoptraill/self.leverage)):
                    
                    print(i,j,'  ', self.pricestoptraill/(self.lastprice+self.lastprice*self.Commission),
                          ' >= ',(1+(self.stoptraill/self.leverage)))
                    self.pricestoptraill=self.lastprice
                    traillist.append(self.pricestoptraill)
#         print(self.cash,self.value)
        return 0

    def run(self,i,j):
        global result_value
        global status_signal
        global result_lose
        global result_profit
        global result_order
        global price
        global startfinnish
        global queue
        self.actionprice=price
        print ('run a backtest for' , i , j)
        startfinnish.append(([i,j],'-'))
        if self.BUY==True:
            if self.datasignal.iloc[-1,0]==1: # 1==buy
                self.size=self.sizer()
                if int(self.actionprice*self.size)<=int(self.cash):
                    self.buy(i,j)

                else:

                    print( '********************** no enough cash for buy **********************' )
                    print(i,self.size,self.actionprice*self.size,'<=',self.cash)

        if self.SELL==True:
            if self.datasignal.iloc[-1,0]==0: # 0==sell
                self.size=self.sizer(self.actionprice)
                if int(self.actionprice*self.size)<=int(self.cash) :
                    self.sell(i,j)

                else:
                    print( '********************** no enough cash for sell **********************' )
                    print(i,self.size,self.actionprice*self.size,'<=',self.cash)

        if self.leverage>1:
            print('value with leverage: ',self.value)
            self.value=self.value-self.leveragevalue
        print('finish11',i,j)

        result_value[i,j]=self.value
        startfinnish.append(('-',[i,j]))
        with open("startfinnish.txt", "w") as output:
            output.write(str(startfinnish)+'\n')


        self.order=pd.DataFrame(self.order)
        self.order.columns = self.order.iloc[0] #set the first row as the df header
        self.order = self.order[1:] #take the data less the header row
        self.profit=pd.DataFrame(self.profit)
        self.profit.columns = self.profit.iloc[0]
        self.profit = self.profit[1:]
        self.lose=pd.DataFrame(self.lose)
        self.lose.columns = self.lose.iloc[0]
        self.lose = self.lose[1:]
        # قفل کردن این ترد برای همزمان نشدن استفاده از فایلها توسط ترد های دیگه
        ui=1
        while ui==1:
            if queue==0:
                queue=1
                break
            time.sleep(1)

        if type(result_order.loc[i,j])==float:
            result_order.at[i,j]=self.order.copy()
        if type(result_profit.loc[i,j])==float:
            result_profit.at[i,j]=self.profit.copy()
        if type(result_lose.loc[i,j])==float:
            result_lose.at[i,j]=self.lose.copy()

        result_order.at[i,j]=result_order.loc[i,j].append(self.order).copy()
        result_profit.at[i,j]=result_profit.loc[i,j].append(self.profit).copy()
        result_lose.at[i,j]=result_lose.loc[i,j].append(self.lose).copy()

        result_order.to_pickle("result_order.pkl")
        result_profit.to_pickle("result_profit.pkl")
        result_lose.to_pickle("result_lose.pkl")
        result_value.to_pickle("result_value.pkl")
        status_signal.at[i,j]=0
        queue=0
        return
#.......................................................................................................................

inde=sellorbuy.index
column=sellorbuy.columns

status_signal=pd.DataFrame(0,index=inde,columns=column) # برای ذخیره  هر اندیکاتور که در حال معامله هست یا نه
try:
    result_order=pd.read_pickle("result_order.pkl")
    result_profit=pd.read_pickle("result_profit.pkl")
    result_lose=pd.read_pickle("result_lose.pkl")
    result_value=pd.read_pickle("result_value.pkl")
    text_file = open("startfinnish.txt", "r")
    startfinnish=list(eval(text_file.readlines()[0]))
    
    except Exception as error:
        print('oooooooooooooo')
        result_value=pd.DataFrame(100,index=inde,columns=column)
        result_order=pd.DataFrame(index=inde,columns=column)
        result_profit=pd.DataFrame(index=inde,columns=column)
        result_lose=pd.DataFrame(index=inde,columns=column)
        startfinnish=[('start','finnish')]

x=1
queue=0
numbertrade=0
# def main()
while x==1:

    if chprice==0:
        thread=threading.Thread(target=runprice)
        thread.start()
    time_delta=timedif()
    if time_delta!='0:00':
#         sellorbuy=runmain() #محاسبه اندیکاتورها
        p = multiprocessing.Pool(1)
        sellorbuy = p.apply_async(runmain).get()
        p.close()
        p.join()
        print('get data')
        for j in sellorbuy.columns:
            for i in sellorbuy.index:
                if type(sellorbuy.loc[i,j])==int:
                    continue
                if (sellorbuy.loc[i,j].iloc[-1,0]== 0 or sellorbuy.loc[i,j].iloc[-1,0]== 1) and status_signal.loc[i,j]==0:
                    numbertrade+=1
                    with open("numbertrade.txt", "w") as output:
                        output.write(str(numbertrade)+'\n')
                    print('status_signal',status_signal.at[i,j] , 'for',i,j,'\n')
                    datasignal=pd.DataFrame(sellorbuy.loc[i,j].copy())
                    status_signal.at[i,j]=1
                    print(status_signal.at[i,j],'\n')

                    bt=backtest(datasignal=datasignal,total_size=True,value=result_value.loc[i,j],
                    trigger='stoptraill',Commission=0.002,stoptraill=0.05,sl=0.04,leverage=6)

                    thread=threading.Thread(target=bt.run,args=(i,j))
                    thread.start()

In [ ]:
# print(sys.getrefcount(bt))
import gc
# gc.get_objects()
for i in gc.get_objects(): 
    print(i)

In [1]:
import numpy as np
sl=list(np.around(np.array(np.arange(0.01, 0.05, 0.01).tolist()),2))
stoptraill=list(np.around(np.array(np.arange(0.01, 0.25, 0.01).tolist()),2))
leverage=list(np.around(np.array(np.arange(1, 15, 1).tolist()),2))
print(len(sl))
print(len(stoptraill))
print(len(leverage))

4
24
14


In [4]:
import pandas as pd
pairs = pd.read_pickle("pairs_kucoin_future.pkl")
# pairs = pairs.loc[[0]]
pairs#.loc[0,'pair']
pd.read_pickle("data_pairs_kucoin_future_1m.pkl").loc[0,'pair']#.iloc[-1,1]
# pairs.to_pickle("pairs_kucoin_future.pkl")

,date,time,open,high,low,close,volume
0,17/07/2022,10:03:00,21300.0,21308.0,21290.0,21308.0,15127
1,17/07/2022,10:04:00,21305.0,21306.0,21287.0,21294.0,18838
2,17/07/2022,10:05:00,21294.0,21311.0,21293.0,21309.0,20163
3,17/07/2022,10:06:00,21308.0,21309.0,21296.0,21299.0,18108
4,17/07/2022,10:07:00,21299.0,21308.0,21299.0,21307.0,7453
...,...,...,...,...,...,...,...
495,17/07/2022,18:18:00,21186.0,21199.0,21185.0,21194.0,23125
496,17/07/2022,18:19:00,21194.0,21195.0,21190.0,21192.0,15990
497,17/07/2022,18:20:00,21192.0,21192.0,21187.0,21187.0,42368
498,17/07/2022,18:21:00,21187.0,21204.0,21187.0,21202.0,16999


In [2]:
from ohlcv import ohlcvs
import asyncio
import nest_asyncio
import os
nest_asyncio.apply()
d=ohlcvs()
asyncio.run(d.run_kucoin_future('1m',500))
# asyncio.run(d.run_binance_margin('15m',400))
pd.read_pickle("data_pairs_kucoin_future_1m.pkl").loc[0,'pair']#.iloc[-1,1]

<Response [200]>
<Response [200]>
<Response [200]>
get ohlcv


NameError: name 'pd' is not defined